# Visualización de precipitaciones en un periodo determinado

Objetivo: Generar un mapa donde se visualice la estimación de lluvias acumuladas durante un período de tiempo en el área de influencia de la Bolsa de Cereales de Bahía Blanca.

Documentación del sensor utilizado:
* CHIRPS (Climate Hazards Group Infrared Precipitation with Station Data) diario: https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY


## Importamos librerías necesarias y nos logueamos en GEE

In [1]:
#Importación de librerías
import ee

In [2]:
#Nos autenticamos en Google Earth Engine
!earthengine authenticate
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=xo5CYzhlPri4SlPtglAnLbxgH7mgUV6Dypda_jK70CQ&tc=Kr8xRI-7MJwXSMaiH8qrhh5j_rQA-V-lMpWk5jje2jA&cc=k_elG6ukOJm6D9vXGrUDkA2VaniJgxZMGpaoFQFKVwU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk5JUohP0qTrWhAMKBVvu_cX56RLAtVUS6YBwqcBxSNyEERhh2Q6Bns

Successfully saved authorization token.


In [3]:
#Nos logueamos al Google Drive
from google.colab import drive # La usamos para montar nuestra unidad de Google Drive
drive.mount('/content/drive') # Montamos nuestra unidad de Google Drive

Mounted at /content/drive


In [4]:
#Instalamos e importamos GEEmap para visualizar la información geográfica en un mapa
!pip install geemap
import geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=ff38f4c52143ec561c8584f399c90a8e84b5c22312925a13f3ac7

## Abrimos un mapa satelital de base

In [6]:
mapa = geemap.Map()

#Seteamos la vista de fondo en "satelital"
mapa.setOptions('SATELLITE')

mapa

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Cargamos los shape de la región y zonas/subzonas

In [7]:
#Cargamos el shape de la region BCP luego de almacenarlo en la cuenta de GEE
area_bcp = ee.FeatureCollection('users/jjablasco/BCP/area_bcp')
mapa.addLayer(area_bcp, {'color': 'red'})

In [ ]:
#Cargamos el shape de la zona norte luego de almacenarlo en la cuenta de GEE
norte = ee.FeatureCollection('users/jjablasco/BCP/norte')
mapa.addLayer(norte, {'color': 'purple'})

In [ ]:
#Cargamos el shape de la zona centro luego de almacenarlo en la cuenta de GEE
centro = ee.FeatureCollection('users/jjablasco/BCP/centro')
mapa.addLayer(centro, {'color': 'green'})

In [ ]:
#Cargamos el shape de la sur norte luego de almacenarlo en la cuenta de GEE
sur = ee.FeatureCollection('users/jjablasco/BCP/sur')
mapa.addLayer(sur, {'color': 'yellow'})

In [ ]:
#Shape de prueba del partido de Trenque Lauquen
trenque = ee.FeatureCollection('users/jjablasco/BCP/trenque')
mapa.addLayer(trenque, {'color': 'orange'})

### Definimos el AOI con el que trabajaremos y lo visualizamos

In [8]:
aoi = area_bcp #modificamos en función de lo necesario

#Hacemos zoom sobre la capa shape previamente cargada
mapa.centerObject(aoi, 6)   #cambiamos la escala de acuerdo al AOI
mapa

Map(bottom=754.0, center=[-37.30624194547664, -62.52767579673791], controls=(ZoomControl(options=['position', …

## Importamos las imágenes CHIRPS precipitation en periodo de cultivo

In [9]:
#Importamos la colección CHIRPS precipitation
chirpsPRECIP = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').select('precipitation')

## Acumulado de precipitaciones

In [10]:
#Filtrar desde el mes de octubre hasta marzo incluido para el ciclo de cultivo
fecha_inicio = '2022-10-01'
fecha_final = '2023-03-01'
precip_gir_suma = chirpsPRECIP.filterDate(fecha_inicio, fecha_final).sum().clip(aoi)

### Visualizar la imagen de precipitacion acumulada durante el cultivo

In [14]:
#Definimos una paleta de colores HEX para el mapa
vis_params = {'min': 100, 'max': 450, 'palette': ['d1ecff','9ed5ff','80d0ff','00c4fa','009bf5','007deb','005cf0','001cbd','001c80']}

#Añadimos la capa de visualización a GEE
mapa.addLayer(precip_gir_suma, vis_params, 'Acumulado de precipitaciones')
mapa

Map(bottom=10324.0, center=[-37.30624194547664, -62.52767579673791], controls=(ZoomControl(options=['position'…

### Exportamos el mapa resultante a nuestro Drive

In [ ]:
exportar_mapa = ee.batch.Export.image.toDrive(
  image= precip_gir_suma,           #Modificar de acuerdo a la layer y AOI que querramos exportar
  description= 'precip_sum',        #Editamos el nombre con el que se guardara en nuestro Drive
  scale= 10,
  region= aoi.geometry(),
  maxPixels= 1e13,
  crs='EPSG:32720')

exportar_mapa.start()

In [ ]:
print('Task Status:', exportar_mapa.status()['state']) # Chequeamos el estado de la exportacion al drive

Task Status: RUNNING


## Anomalía de precipitaciones

In [15]:
#Filtrar desde el mes de octubre hasta marzo incluido para el ciclo de cultivo
precip_hist = chirpsPRECIP.filter(ee.Filter.calendarRange(10,3,'month')).filterDate('2002-10-01','2023-03-31')

In [16]:
#Filtramos la fecha al ciclo del cultivo
fecha_inicio = ee.Date('2022-10-01')
fecha_final = ee.Date('2023-03-31')
precip_gir_actual = chirpsPRECIP.filterDate(fecha_inicio, fecha_final).mean()

In [17]:
precip_prom = precip_hist.mean()                        #Promedio
precip_std = precip_hist.reduce(ee.Reducer.stdDev())    #Desvío estandar

### Calcular la anomalía estándarizada de precipitaciones para el ciclo de cultivo

In [18]:
precip_anom_std = precip_gir_actual.subtract(precip_prom).divide(precip_std).clip(aoi)

### Visualizar la imagen de anomalía de precipitaciones

In [19]:
#Definimos una paleta de colores para el mapa
vis_params = {'min': -0.15, 'max': 0.15, 'palette': ['4d1103','992206','e43309','e4a20c','c8c8c8','c1ff89','5bff18','379b0e','13580b']}

#Añadimos la capa de visualización a GEE
mapa.addLayer(precip_anom_std, vis_params, 'Anomalía estándarizada NDVI')
mapa

Map(bottom=10324.0, center=[-37.30624194547664, -62.52767579673791], controls=(ZoomControl(options=['position'…

### Exportamos el mapa resultante a nuestro Drive

In [ ]:
exportar_mapa = ee.batch.Export.image.toDrive(
  image= precip_anom_std,           #Modificar de acuerdo a la layer y AOI que querramos exportar
  description= 'precip_anom',    #Editamos el nombre con el que se guardara en nuestro Drive
  scale= 10,
  region= aoi.geometry(),
  maxPixels= 1e13,
  crs='EPSG:32720')

exportar_mapa.start()

In [ ]:
print('Task Status:', exportar_mapa.status()['state']) # Chequeamos el estado de la exportacion al drive